In [1]:
# check GPU
!nvidia-smi

Wed Jul  7 10:05:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import sys

MODULE_PATH = '/content/drive/MyDrive/GitHub/DL_Study/CNN'

sys.path.insert(0, MODULE_PATH)
sys.path

['/content/drive/MyDrive/GitHub/DL_Study/CNN',
 '',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [3]:
# import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import numpy

from ShuffleNet import *

torch.manual_seed(42)

# for time series split
!pip install scikit-learn==0.24.2

     |████████████████████████████████| 22.3MB 1.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [4]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device

def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

def np_to_tensor(data):
    device = get_device()
    return torch.tensor(data).float().to(device)

# configuration setting
def model_config():
    # parameter for CNN Model
    epochs = [30]
    batch_size = [64]
    learning_rate = [0.01, 0.001]
    
    # create config data
    configs = []
    for i in epochs:
        for j in batch_size:
            for k in learning_rate:
                config = [i, j, k]
                configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config, verbose=0):

    # unpack config
    n_epochs, n_batch, learning_rate = config
    # use ShuffleNet for CNN
    model = ShuffleNet(groups=3, in_channels=1)
    if torch.cuda.is_available():
        model.cuda()

    # define Loss and Optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    data_size = train_X.size(0)
    max_iters = data_size//n_batch

    for epoch in range(1, n_epochs+1):
        #shuffle data
        idx = numpy.random.permutation(numpy.arange(data_size))
        x_data = train_X[idx]
        y_data = train_y[idx]

        epoch_loss = 0
        start_time = time.time()
        for it in range(max_iters):
            batch_x = x_data[it*n_batch:(it+1)*n_batch]
            batch_y = y_data[it*n_batch:(it+1)*n_batch]

            optimizer.zero_grad()
            predict = model(batch_x)
            loss = criterion(predict, batch_y)
            loss.backward()
            optimizer.step()

            epoch_loss+= loss.item()
        avg_loss = epoch_loss/max_iters

        if verbose:
            duration = start_time-time.time()
            print(f'epoch:{epoch}/{epochs}, ì‹œê°„:{duration:.2f}[s], loss:{avg_loss:.5f}')


    return model

def MAE_metric(x, t):
    t = np.array(t)
    return np.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    t = np.array(t)
    return np.mean((x-t)**2)

In [5]:
# dataset
import pandas as pd
import numpy as np
import numpy
import time
from datetime import datetime

np.random.seed(42)
numpy.random.seed(42)

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00492/Metro_Interstate_Traffic_Volume.csv.gz'
df = pd.read_csv(data_url,compression='gzip', index_col=7)
df.drop(columns='weather_description', inplace=True)
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,traffic_volume
date_time,,,,,,,
2012-10-02 09:00:00,None,288.28,0.0,0.0,40,Clouds,5545
2012-10-02 10:00:00,None,289.36,0.0,0.0,75,Clouds,4516
2012-10-02 11:00:00,None,289.58,0.0,0.0,90,Clouds,4767
2012-10-02 12:00:00,None,290.13,0.0,0.0,90,Clouds,5026
2012-10-02 13:00:00,None,291.14,0.0,0.0,75,Clouds,4918


In [6]:
from scipy.stats import skew, kurtosis
from statsmodels.tsa.stattools import adfuller

# jb = (n/6)*(skewness**2 + (kurtosis**2/4))

def data_statistics(df):
    df = df.dropna()
    data = df.values
    num = len(df)
    skewness_ = skew(data)
    kurtosis_ = kurtosis(data)
    jarque_bera_ = (num/6)*(skewness_**2 + (kurtosis_**2/4))
    result = adfuller(data)
    adf_ = result[0]
    print(f'skewness : {skewness_}')
    print(f'kurtosis : {kurtosis_}')
    print(f'jarque bera : {jarque_bera_}')
    print(f'ADF : {adf_}')

data_statistics(df['traffic_volume'])

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


skewness : -0.08937857006592761
kurtosis : -1.3090768978229412
jarque bera : 3506.110788859986
ADF : -28.016624319503496


In [7]:
df.describe()

,temp,rain_1h,snow_1h,clouds_all,traffic_volume
count,48204.000000,48204.000000,48204.000000,48204.000000,48204.000000
mean,281.205870,0.334264,0.000222,49.362231,3259.818355
std,13.338232,44.789133,0.008168,39.015750,1986.860670
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,272.160000,0.000000,0.000000,1.000000,1193.000000
50%,282.450000,0.000000,0.000000,64.000000,3380.000000
75%,291.806000,0.000000,0.000000,90.000000,4933.000000
max,310.070000,9831.300000,0.510000,100.000000,7280.000000


In [8]:
df.isnull().sum()

holiday           0
temp              0
rain_1h           0
snow_1h           0
clouds_all        0
weather_main      0
traffic_volume    0
dtype: int64

In [9]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

from sklearn.model_selection import TimeSeriesSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder

dataset = df.values
holiday_encoder = LabelEncoder()
weather_encoder = ColumnTransformer(
    [('1', OneHotEncoder(), [5])],
    remainder='passthrough'
)
dataset[:, 0] = holiday_encoder.fit_transform(dataset[:, 0])
dataset = weather_encoder.fit_transform(dataset)
dataset = pd.DataFrame(dataset)
cols = dataset.columns.tolist()
cols = cols[-1:]+cols[:-1]
dataset = dataset[cols]
dataset = dataset.values
dataset = dataset.astype('float')

n_inputs = 24
n_features = 17
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)
new_df.head()

,var1(t-24),var2(t-24),var3(t-24),var4(t-24),var5(t-24),var6(t-24),var7(t-24),var8(t-24),var9(t-24),var10(t-24),var11(t-24),var12(t-24),var13(t-24),var14(t-24),var15(t-24),var16(t-24),var17(t-24),var1(t-23),var2(t-23),var3(t-23),var4(t-23),var5(t-23),var6(t-23),var7(t-23),var8(t-23),var9(t-23),var10(t-23),var11(t-23),var12(t-23),var13(t-23),var14(t-23),var15(t-23),var16(t-23),var17(t-23),var1(t-22),var2(t-22),var3(t-22),var4(t-22),var5(t-22),var6(t-22),...,var13(t-3),var14(t-3),var15(t-3),var16(t-3),var17(t-3),var1(t-2),var2(t-2),var3(t-2),var4(t-2),var5(t-2),var6(t-2),var7(t-2),var8(t-2),var9(t-2),var10(t-2),var11(t-2),var12(t-2),var13(t-2),var14(t-2),var15(t-2),var16(t-2),var17(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),var11(t-1),var12(t-1),var13(t-1),var14(t-1),var15(t-1),var16(t-1),var17(t-1),var1(t)
24,5545.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,288.28,0.0,0.0,40.0,4516.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,289.36,0.0,0.0,75.0,4767.0,0.0,1.0,0.0,0.0,0.0,...,7.0,278.23,0.0,0.0,1.0,6511.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,278.12,0.0,0.0,1.0,5471.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,282.48,0.0,0.0,1.0,5097.0
25,4516.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,289.36,0.0,0.0,75.0,4767.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,289.58,0.0,0.0,90.0,5026.0,0.0,1.0,0.0,0.0,0.0,...,7.0,278.12,0.0,0.0,1.0,5471.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,282.48,0.0,0.0,1.0,5097.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,291.97,0.0,0.0,1.0,4887.0
26,4767.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,289.58,0.0,0.0,90.0,5026.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,290.13,0.0,0.0,90.0,4918.0,0.0,1.0,0.0,0.0,0.0,...,7.0,282.48,0.0,0.0,1.0,5097.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,291.97,0.0,0.0,1.0,4887.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,293.23,0.0,0.0,1.0,5337.0
27,5026.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,290.13,0.0,0.0,90.0,4918.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,291.14,0.0,0.0,75.0,5181.0,1.0,0.0,0.0,0.0,0.0,...,7.0,291.97,0.0,0.0,1.0,4887.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,293.23,0.0,0.0,1.0,5337.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,294.31,0.0,0.0,1.0,5692.0
28,4918.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,291.14,0.0,0.0,75.0,5181.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,291.72,0.0,0.0,1.0,5584.0,1.0,0.0,0.0,0.0,0.0,...,7.0,293.23,0.0,0.0,1.0,5337.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,294.31,0.0,0.0,1.0,5692.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,295.17,0.0,0.0,1.0,6137.0


In [10]:
n_splits = 10
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs).split(new_df)
next(train_test_split)

configs = model_config()
history = []

best_rmse, best_mse, best_mae = [], [], []
learning_time = []

i = 1

print('config : epochs, batch_size, learning_rate')

# nested cross validation for time series model
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = int(len(train_cv_X)*0.2)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1, 1, n_inputs, n_features)
    val_X = val_X.reshape(-1, 1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1, 1, n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, 1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    train_X = np_to_tensor(train_X)
    train_y = np_to_tensor(train_y)
    val_X = np_to_tensor(val_X)
    val_y = np_to_tensor(val_y)
    train_cv_X = np_to_tensor(train_cv_X)
    train_cv_y = np_to_tensor(train_cv_y)
    test_cv_X = np_to_tensor(test_cv_X)
    test_cv_y = np_to_tensor(test_cv_y)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        predicted = model(val_X)
        error = F.mse_loss(predicted, val_y)   # rmse
        print(f'error(rmse):{error.item():.2f}')
        if errors:
            if error.item() < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error.item())

    history.append(errors)

    # outer
    start_time = time.time()
    # model fitting
    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    # check time
    duration = time.time() - start_time
    predicted = selected_model(test_cv_X)
    rmse = np.sqrt(F.mse_loss(predicted, test_cv_y).item())
    mse = F.mse_loss(predicted, test_cv_y)
    mae = F.l1_loss(predicted, test_cv_y)
    best_rmse.append(rmse)
    best_mse.append(mse.item())
    best_mae.append(mae.item())
    learning_time.append(duration)

    # model eval
    print(f'train-size:{train_X.size(0)}, val-size:{val_X.size(0)}, test-size:{test_cv_X.size(0)}')
    print(f'best_model => error(rmse) : {rmse:.2f}, param:{configs[param]}, times: {duration:.3f}')
    print()

config : epochs, batch_size, learning_rate
fold : 1/10
 == train [30, 64, 0.01] model ==  

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


error(rmse):1032672.06
 == train [30, 64, 0.001] model ==  error(rmse):2396606.75
train-size:6405, val-size:1601, test-size:4015
best_model => error(rmse) : 1256.36, param:[30, 64, 0.01], times: 122.144

fold : 2/10
 == train [30, 64, 0.01] model ==  error(rmse):1261518.50
 == train [30, 64, 0.001] model ==  error(rmse):1524947.75
train-size:9617, val-size:2404, test-size:4015
best_model => error(rmse) : 944.87, param:[30, 64, 0.01], times: 180.437

fold : 3/10
 == train [30, 64, 0.01] model ==  error(rmse):1111582.00
 == train [30, 64, 0.001] model ==  error(rmse):1290826.25
train-size:12829, val-size:3207, test-size:4015
best_model => error(rmse) : 1103.49, param:[30, 64, 0.01], times: 244.094

fold : 4/10
 == train [30, 64, 0.01] model ==  error(rmse):1126272.50
 == train [30, 64, 0.001] model ==  error(rmse):1809993.88
train-size:16041, val-size:4010, test-size:4015
best_model => error(rmse) : 1111.96, param:[30, 64, 0.01], times: 301.793

fold : 5/10
 == train [30, 64, 0.01] model

In [11]:
predicted = selected_model(test_cv_X)

def model_evaluation(mse, rmse, mae):
    mse = np.array(mse)
    rmse = np.array(rmse)
    mae = np.array(mae)
    print(f'MSE: mean={np.mean(mse)}, std={np.std(mse)}')
    print(f'RMSE: mean={np.mean(rmse)}, std={np.std(rmse)}')
    print(f'MAE: mean={np.mean(mae)}, std={np.std(mae)}')

model_evaluation(best_mse, best_rmse, best_mae)

# check time
print()
print('[training time]')
print(f'mean : {np.mean(np.array(learning_time))}, last:{learning_time[-1]}')

MSE: mean=896511.871875, std=506028.6048780697
RMSE: mean=908.4075277206712, std=267.03489557624914
MAE: mean=603.93115234375, std=176.15203875476

[training time]
mean : 395.39701623916625, last:662.9773278236389
